In [1]:
import os
import sys
sys.path.append('../../')
import math
import pandas as pd
import numpy as np
import recs.path as path
import json
from recs.backtest import get_backtest_answer, model_score_logger
from recs.cf_mf import MatrixFactorization, generate_cf_predict_df
from recs.util import id2cat, recall_evaluate, readjson2dict
import recs.path as path
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from tqdm import tqdm
import time

In [2]:
# Date Setting
pred_period = [202001, 202007]
history_period = [201600, 202000]
best_sell_period = [201800 ,202000]
current_yyyymm = 201912

# Date Validations
if (history_period[1] > pred_period[0]):
    raise SystemExit("Error: predict_start > history_end")
    
if (current_yyyymm > history_period[1]):
    raise SystemExit("Error: target_user_date > history_end")

u2idx = readjson2dict("crm_idx")
i2idx = readjson2dict("fund_info_idx")

crm_path = os.path.join(path.data_path, "crm_diff.pkl")
trans_buy_path = os.path.join(path.data_path, "trans_buy.pkl")


trans_buy = pd.read_pickle(trans_buy_path)
crm = pd.read_pickle(crm_path)
target_users = crm[crm.yyyymm == current_yyyymm].id_number.unique().tolist()

# training
train_df = trans_buy[
    (trans_buy.buy_date < history_period[1]*100) &
    (trans_buy.buy_date > history_period[0]*100) 
]

# testing
test_df = trans_buy[
    (trans_buy.buy_date > pred_period[0]*100) &
    (trans_buy.buy_date < pred_period[1]*100)
]

# Testing Result

test_users = test_df.id_number.unique().tolist()

test = get_backtest_answer(test_df, test_users, pred_period[0]*100, pred_period[1]*100)


100%|██████████| 24460/24460 [00:44<00:00, 548.48it/s]


In [45]:
tqdm.pandas()
train_df['t_weight'] = train_df.buy_date.progress_apply(lambda x: (x - train_df.buy_date.min() + 1) / (train_df.buy_date.max() - train_df.buy_date.min()))

100%|██████████| 233507/233507 [02:52<00:00, 1351.19it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [60]:
rate_df = train_df.groupby(["id_number", "fund_id"]).agg({'t_weight':'sum'}).reset_index().rename(columns={"t_weight": "Rate"})
# rate_df = train_df.groupby(["id_number", "fund_id"]).size().reset_index(name="Rate")
best_sells = trans_buy[
    (trans_buy.buy_date > best_sell_period[0]*100) & (trans_buy.buy_date < best_sell_period[1]*100)
].fund_id.value_counts().rename_axis('fund_id').reset_index(name='count')

def rec_forward(rate_df, svds_k, target_users = test_users, ignore_item=[], train_bs_funds=200):
    rate_df = rate_df[~rate_df.fund_id.isin(ignore_item)]
    rate_df =rate_df[
        rate_df['fund_id'].isin(best_sells['fund_id'].tolist()[:train_bs_funds])
    ]
    cf_preds_matrix = generate_cf_predict_df(rate_df, svds_k)
    CF = MatrixFactorization(cf_preds_matrix)
        
    rec_dict = {}
    for u in tqdm(target_users):
        ignore_items = train_df[train_df.id_number == u].fund_id.unique().tolist()
        if (u in cf_preds_matrix.columns.tolist()):            
            rec_dict[u] = CF.rec_items(u, items_to_ignore=ignore_items).fund_id.tolist()
        else:
            best_sell_funds = best_sells[~best_sells['fund_id'].isin(ignore_items)]['fund_id'].tolist()[:10]
            rec_dict[u] = best_sell_funds
            
    return rec_dict
    

In [53]:
params = {
    "svd_k": 15,
}
rec_dict = rec_forward(rate_df, params["svd_k"])
scores = recall_evaluate(rec_dict, test)
model_score_logger('MF', params, scores)
print(scores)

100%|██████████| 24460/24460 [00:00<00:00, 215018.58it/s]

{'recall@3': 0.029715549271708684, 'recall@5': 0.04011174825264121, 'recall@10': 0.055852809816294875, 'recall@all': 0.055852809816294875}


In [66]:
for i in range(20):
    for k in range(8):
        print(i)
        params = {
            "train_bs_funds": (i+1)*100,
            "svd_k": ((i+1)*100*(k+1))//10
        }
        rec_dict = rec_forward(rate_df, params["svd_k"], train_bs_funds=params["train_bs_funds"])
        scores = recall_evaluate(rec_dict, test)
        model_score_logger('MF', params, scores)

0


100%|██████████| 24460/24460 [00:00<00:00, 214953.71it/s]


0


100%|██████████| 24460/24460 [00:00<00:00, 203753.37it/s]


0


...

100%|██████████| 24460/24460 [00:00<00:00, 215183.64it/s]


19


100%|██████████| 24460/24460 [00:00<00:00, 213528.78it/s]


19


100%|██████████| 24460/24460 [00:00<00:00, 209907.86it/s]
